ColBERT & DeepCT

In [1]:
!pip install tensorflow==1.15
!pip install -U python-terrier
!pip install -U git+https://github.com/terrierteam/pyterrier_colbert
!pip install -U git+https://github.com/terrierteam/pyterrier_deepct.git
!pip install transformers==3.0.2
!pip install faiss-gpu==1.6.3

     |████████████████████████████████| 412.3 MB 10 kB/s 
     |████████████████████████████████| 3.8 MB 42.6 MB/s 
     |████████████████████████████████| 503 kB 41.0 MB/s 
     |████████████████████████████████| 50 kB 6.7 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=90e8f7e4d2d6552189c4e3442e076b68360bbd4e8ee40b5ce086d5f139f9113d
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.6.0
    Uninstalling tensorflow-estimator-2.6.0:
      Successfully uninstalled tensorflow-estimator-2.6.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.6.0
    Uninstalling tensorboard-2.6.0:
      Successfully uninstalled tensorboard-2.6.0
  Attempting uninstall: gast
    Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Succes

In [2]:
import pyterrier as pt
import os
import pandas as pd
import pyterrier_colbert.ranking
import pyterrier_colbert.indexing
import pyterrier_deepct
import tensorflow as tf
import sys
import faiss

if not pt.started():
    pt.init()

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
tf.logging.set_verbosity(tf.logging.ERROR)

terrier-assemblies 5.6  jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.6  jar not found, downloading to /root/.pyterrier...
Done
PyTerrier 0.7.0 has loaded Terrier 5.6 (built by craigmacdonald on 2021-09-17 13:27)


In [3]:
def experiment_br(retrs, topics, qrels, labels):
  return pt.Experiment(retrs, topics, qrels, names=labels, eval_metrics=['map', 'ndcg_cut.10', 'P.10', 'mrt'])

In [4]:
"""
!rm -rf /content/sample_data
!rm -rf /content/index_vaswani
!rm -rf /content/index_colbert
"""

'\n!rm -rf /content/sample_data\n!rm -rf /content/index_vaswani\n!rm -rf /content/index_colbert\n'

In [5]:
dataset_name = 'irds:vaswani'
path_index_vaswani = '/content/index_vaswani'
props_path = '/data.properties'
dataset = pt.datasets.get_dataset(dataset_name)

if not os.path.exists(path_index_vaswani):
    indexer = pt.index.IterDictIndexer(path_index_vaswani)
    index_ref = indexer.index(dataset.get_corpus_iter(), meta=('docno', 'text'))
else:
    index_ref = pt.IndexRef.of(path_index_vaswani + props_path)

index = pt.IndexFactory.of(index_ref)
topics = dataset.get_topics()
qrels = dataset.get_qrels()
evals = ['map', 'ndcg_cut.10', 'P.10', 'mrt']
br = pt.BatchRetrieve(index, wmodel='BM25')

[INFO] [starting] building docstore
[INFO] [starting] http://ir.dcs.gla.ac.uk/resources/test_collections/npl/npl.tar.gz
docs_iter:   0%|                                     | 0/11429 [00:00<?, ?doc/s]
http://ir.dcs.gla.ac.uk/resources/test_collections/npl/npl.tar.gz: 0.0%| 0.00/2.13M [00:00<?, ?B/s]
http://ir.dcs.gla.ac.uk/resources/test_collections/npl/npl.tar.gz: 2.3%| 49.2k/2.13M [00:00<00:08, 255kB/s]
http://ir.dcs.gla.ac.uk/resources/test_collections/npl/npl.tar.gz: 4.6%| 98.3k/2.13M [00:00<00:06, 331kB/s]
http://ir.dcs.gla.ac.uk/resources/test_collections/npl/npl.tar.gz: 7.7%| 164k/2.13M [00:00<00:04, 408kB/s] 
http://ir.dcs.gla.ac.uk/resources/test_collections/npl/npl.tar.gz: 15.8%| 336k/2.13M [00:00<00:02, 664kB/s]
http://ir.dcs.gla.ac.uk/resources/test_collections/npl/npl.tar.gz: 32.0%| 680k/2.13M [00:00<00:01, 1.11MB/s]

[INFO] [finished] http://ir.dcs.gla.ac.uk/resources/test_collections/npl/npl.tar.gz: [00:00] [2.13MB] [2.88MB/s]
docs_iter:   0%|                            

vaswani documents:   0%|          | 0/11429 [00:00<?, ?it/s]

ColBERT

In [6]:
checkpoint_colbert = 'http://www.dcs.gla.ac.uk/~craigm/colbert.dnn.zip'
path_index_colbert = '/content/index_colbert'

if not os.path.exists(path_index_colbert):
    indexer_colbert = pyterrier_colbert.indexing.ColBERTIndexer(checkpoint_colbert, '/content', 'index_colbert', chunksize=3)
    indexer_colbert.index(dataset.get_corpus_iter())

colbert_factory = pyterrier_colbert.ranking.ColBERTFactory(checkpoint_colbert, None, None)
colbert = colbert_factory.text_scorer(doc_attr='text')
pipeline_colbert = br >> pt.text.get_text(dataset, 'text') >> colbert

vaswani documents:   0%|          | 0/11429 [00:00<?, ?it/s]

[Sep 23, 19:26:01] [0] 		 #> Local args.bsize = 128
[Sep 23, 19:26:01] [0] 		 #> args.index_root = /content
[Sep 23, 19:26:01] [0] 		 #> self.possible_subset_sizes = [69905]


[INFO] Lock 140369738039248 acquired on /root/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517.lock
[INFO] https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json not found in cache or force_download set to True, downloading to /root/.cache/torch/transformers/tmp6cf81bb2


Downloading:   0%|          | 0.00/433 [00:00<?, ?B/s]

[INFO] storing https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json in cache at /root/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
[INFO] creating metadata file for /root/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
[INFO] Lock 140369738039248 released on /root/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517.lock
[INFO] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /root/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
[INFO] Model config BertConfig {
 

...

/usr/local/lib/python3.7/dist-packages/pyterrier_colbert/ranking.py:232: UserWarning: No index_root and index_name specified - no index ranking possible
  warn("No index_root and index_name specified - no index ranking possible")
[INFO] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /root/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
[INFO] Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "type_vocab_size": 2,
  "

[Sep 23, 19:33:15] #> Loading model checkpoint.
[Sep 23, 19:33:15] #> Loading checkpoint http://www.dcs.gla.ac.uk/~craigm/colbert.dnn.zip


[WARNING] Some weights of the model checkpoint at bert-base-uncased were not used when initializing ColBERT: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing ColBERT from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing ColBERT from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
[WARNING] Some weights of ColBERT were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['linear.weight']
You 

[Sep 23, 19:33:34] #> checkpoint['epoch'] = 0
[Sep 23, 19:33:34] #> checkpoint['batch'] = 44500


[INFO] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /root/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
[INFO] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /root/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


In [7]:
experiment_br([br, pipeline_colbert], topics, qrels, ['BM25', 'ColBERT'])

,name,map,ndcg_cut.10,P.10,mrt
0,BM25,0.296517,0.446609,0.352688,94.376996
1,ColBERT,0.284444,0.430657,0.356989,41296.546267


DEEPCT

In [8]:
if not os.path.exists('marco.zip'):
  !wget http://boston.lti.cs.cmu.edu/appendices/arXiv2019-DeepCT-Zhuyun-Dai/outputs/marco.zip
  !unzip marco.zip

if not os.path.exists('uncased_L-12_H-768_A-12.zip'):
  !wget https://storage.googleapis.com/bert_models/2020_02_20/uncased_L-12_H-768_A-12.zip
  !unzip uncased_L-12_H-768_A-12.zip
  !mkdir -p bert-base-uncased
  !mv vocab.txt bert_* bert-base-uncased/

deepct = pyterrier_deepct.DeepCTTransformer('bert-base-uncased/bert_config.json', 'marco/model.ckpt-65816')
index_loc_deepct = './index_deepct'
indexer_deepct = deepct >> pt.IterDictIndexer(index_loc_deepct)
index_ref_deepct = indexer_deepct.index(dataset.get_corpus_iter())
br_deepct = pt.BatchRetrieve(index_ref_deepct, wmodel="BM25")

--2021-09-23 20:37:44--  http://boston.lti.cs.cmu.edu/appendices/arXiv2019-DeepCT-Zhuyun-Dai/outputs/marco.zip
Resolving boston.lti.cs.cmu.edu (boston.lti.cs.cmu.edu)... 128.2.207.32, 128.2.207.139
Connecting to boston.lti.cs.cmu.edu (boston.lti.cs.cmu.edu)|128.2.207.32|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1237248594 (1.2G) [application/zip]
Saving to: ‘marco.zip’

marco.zip           100%[===================>]   1.15G  78.1MB/s    in 15s     

2021-09-23 20:38:00 (76.3 MB/s) - ‘marco.zip’ saved [1237248594/1237248594]

Archive:  marco.zip
   creating: marco/
  inflating: marco/checkpoint        
  inflating: marco/graph.pbtxt       
  inflating: marco/model.ckpt-65816.data-00000-of-00001  
  inflating: marco/model.ckpt-65816.index  
  inflating: marco/model.ckpt-65816.meta  
  inflating: marco/train.tf_record   
--2021-09-23 20:38:30--  https://storage.googleapis.com/bert_models/2020_02_20/uncased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com

[INFO] NumExpr defaulting to 2 threads.


vaswani documents:   0%|          | 0/11429 [00:00<?, ?it/s]

In [9]:
experiment_br([br, br_deepct], topics, qrels, ['BM25', 'DeepCT'])

,name,map,ndcg_cut.10,P.10,mrt
0,BM25,0.296517,0.446609,0.352688,76.524317
1,DeepCT,0.296275,0.443258,0.359140,36.670649


end of fun.